In [ ]:
import pandas as pd
import numpy as np
import json
from collections import Counter
import copy

In [ ]:
commit_sharing = "./20231012_230826_commit_sharings.json"
pull_request_sharing = "./20231012_233628_pr_sharings.json"
issue_sharing = "./20231012_235128_issue_sharings.json"
file_sharings = "./20231012_234250_file_sharings.json"
discussion_sharings = "./20231012_235320_discussion_sharings.json"

path_lista_sinonimos = "./synonyms_for_normalization.txt"

In [ ]:
# arquivo_json = pull_request_sharing

# Abra o arquivo json
with open(commit_sharing, 'r') as f:
    dados1 = json.load(f)

# Abra o arquivo json
with open(pull_request_sharing, 'r') as f:
    dados2 = json.load(f)

# Abra o arquivo json
with open(issue_sharing, 'r') as f:
    dados3 = json.load(f)

# Abra o arquivo json
with open(file_sharings, 'r') as f:
    dados4 = json.load(f)

# Abra o arquivo json
with open(discussion_sharings, 'r') as f:
    dados5 = json.load(f)

jsons = {
    "data1": dados1,
    "data2": dados2,
    "data3": dados3,
    "data4": dados4,
    "data5": dados5
}

# Normalização

In [ ]:
# Abre o arquivo em modo de leitura
with open(path_lista_sinonimos, 'r') as arquivo:
    # Inicializa um dicionário para armazenar os dados
    synonyms = {}

    # Lê cada linha do arquivo
    for linha in arquivo:
        # Divide a linha em chave e valor usando ':' como delimitador
        chave, valor = linha.strip().split('=')

        # Armazena os dados no dicionário (removendo espaços extras)
        synonyms[chave.strip()] = valor.strip().split(',')

In [ ]:

for tabela in jsons:
  for elementos in jsons[tabela]["Sources"]:
    if elementos["RepoLanguage"] == None :
      elementos["RepoLanguage"] = "none"
    else:
      for linguagem, alias in synonyms.items():
        if elementos["RepoLanguage"] in alias:
          elementos["RepoLanguage"] = linguagem

for tabela in jsons:
  for elementos in jsons[tabela]["Sources"]:
    for elemento in elementos["ChatgptSharing"]:
      if "Conversations" in elemento and elemento["Conversations"]:
        for conversation in elemento["Conversations"]:
          if "ListOfCode" in conversation and conversation["ListOfCode"]:
            for snipetType in conversation["ListOfCode"]:
              if snipetType["Type"] == None :
                snipetType["Type"] = "none"
              else:
                for linguagem, alias in synonyms.items():
                  if  snipetType["Type"] in alias:
                    snipetType["Type"] = linguagem



In [ ]:

caminho_arquivo = 'dados.json'

# Escreve o JSON no arquivo
with open(caminho_arquivo, 'w') as arquivo:
  json.dump(jsons, arquivo)

# Extração de dados

In [ ]:
# len(dados["Sources"])
repoLang = []
for tabela in jsons:
  for elementos in jsons[tabela]["Sources"]:
    repoLang.append(elementos["RepoLanguage"])


In [ ]:
contagem = Counter(repoLang)
contagem

contagemDict = dict(contagem)
# contagemDict["None"] = contagemDict[None]
# del contagemDict[None]

In [ ]:
df0 = pd.DataFrame(list(contagemDict.items()), columns=['Language', 'Count'])

In [ ]:
repoByLanguage = {}

for key in contagem.keys():
  repoByLanguage.setdefault(key, [])
  for tabela in jsons:
    for elementos in jsons[tabela]["Sources"]:
      if elementos["RepoLanguage"] == key:
        repoByLanguage[key].append(elementos["RepoName"])


In [ ]:
# repoByLanguage["None"] = repoByLanguage[None]
# del repoByLanguage[None]


In [ ]:
df1 = pd.DataFrame(list(repoByLanguage.items()), columns=['Language', 'Repos'])
# df.to_csv('repoByLanguage.csv', index=False)

In [ ]:
repoByLanguageCount = copy.deepcopy(repoByLanguage)

for chave, lista in repoByLanguage.items():
    repoByLanguage[chave] = list(set(lista))
    repoByLanguageCount[chave] = len(list(set(lista)))

In [ ]:
df2 = pd.DataFrame(list(repoByLanguage.items()), columns=['Language', 'UniqueRepos'])
df3 = pd.DataFrame(list(repoByLanguageCount.items()), columns=['Language', 'UniqueReposCount'])
# df.to_csv('repoByLanguageUnique.csv', index=False)

## CountChat

In [ ]:
chatByLanguage = {}
for tabela in jsons:
  for elementos in jsons[tabela]["Sources"]:
    chatByLanguage.setdefault(elementos["RepoLanguage"], [])
    for elemento in elementos["ChatgptSharing"]:
      chatByLanguage[elementos["RepoLanguage"]].append(elemento["URL"])



In [ ]:
# chatByLanguage["None"] = chatByLanguage[None]
# del chatByLanguage[None]

In [ ]:
df4 = pd.DataFrame(list(chatByLanguage.items()), columns=['Language', 'Chat'])
# df.to_csv('chatByLanguage.csv', index=False)

In [ ]:
chatByLanguageCount = copy.deepcopy(chatByLanguage)
for chave, lista in chatByLanguage.items():
    chatByLanguage[chave] = list(set(lista))
    chatByLanguageCount[chave] = len(list(set(lista)))

In [ ]:
chatByLanguage
df5 = pd.DataFrame(list(chatByLanguage.items()), columns=['Language', 'UniqueChat'])
df6 = pd.DataFrame(list(chatByLanguageCount.items()), columns=['Language', 'UniqueChatCount'])
# df.to_csv('uniqueChatByLanguage.csv', index=False)

## Snipets de Codigo

In [ ]:
snipetsByLanguage = {}
for tabela in jsons:
  for elementos in jsons[tabela]["Sources"]:
    snipetsByLanguage.setdefault(elementos["RepoLanguage"], [])
    for elemento in elementos["ChatgptSharing"]:
      if "Conversations" in elemento and elemento["Conversations"]:
        for conversation in elemento["Conversations"]:
          if "ListOfCode" in conversation and conversation["ListOfCode"]:
            for snipetType in conversation["ListOfCode"]:
              snipetsByLanguage[elementos["RepoLanguage"]].append(snipetType["Type"])



In [ ]:
# snipetsByLanguage["None"] = snipetsByLanguage[None]
# del snipetsByLanguage[None]


In [ ]:
snipetsByLanguage
snipetsByLanguagePercentage = copy.deepcopy(snipetsByLanguage)
snipetsByLanguageTotal = copy.deepcopy(snipetsByLanguage)

In [ ]:

for chave, lista in snipetsByLanguage.items():
    # Calcular a contagem total de elementos na lista atual
    total_elementos_lista = len(lista)

    # Calcular a contagem de elementos na lista atual
    contagem_elementos = Counter(lista)

    # Calcular a porcentagem de ocorrência de cada elemento na lista atual
    snipetsByLanguagePercentage[chave] = {elemento: f'{(ocorrencias / total_elementos_lista) * 100:.3f}' for elemento, ocorrencias in contagem_elementos.items()}
    snipetsByLanguage[chave] = dict(Counter(snipetsByLanguage[chave]))
    snipetsByLanguageTotal[chave] = (sum(Counter(snipetsByLanguage[chave]).values()))

In [ ]:
df7 = pd.DataFrame(list(snipetsByLanguage.items()), columns=['Language', 'SnippetLang'])

df8 = pd.DataFrame(list(snipetsByLanguageTotal.items()), columns=['Language', 'SnippetLangCount'])

df9 = pd.DataFrame(list(snipetsByLanguagePercentage.items()), columns=['Language', 'SnippetLangPercentage'])

# df_concatenado = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8, df9], axis=1)
df_concatenado = pd.merge(df0, df1, on='Language').merge(df2, on='Language').merge(df3, on='Language').merge(df4, on='Language')
df_concatenado = df_concatenado.merge(df5, on='Language').merge(df6, on='Language').merge(df7, on='Language').merge(df8, on='Language').merge(df9, on='Language')
df_concatenado.to_csv('mega_tabela_resultados.csv', index=False)

## Verificar a porcentagem e plotar graficos

In [ ]:
# snipetsByLanguage

linhas = []
for coluna, valores in snipetsByLanguage.items():
    linha = {'linguagem': coluna}  # Adicione a coluna principal
    linha.update(valores)  # Adicione os valores da linguagem
    linhas.append(linha)

# Criação do DataFrame
snipetsByLanguageDF = pd.DataFrame(linhas)
snipetsByLanguageDF = snipetsByLanguageDF.T


# Exibindo o DataFrame
snipetsByLanguageDF = snipetsByLanguageDF.fillna(0)
snipetsByLanguageDF.to_csv('snipetsByLanguageDF.csv')

In [ ]:
# # snipetsByLanguagePercentage

linhas = []
for coluna, valores in snipetsByLanguagePercentage.items():
    linha = {'linguagem': coluna}  # Adicione a coluna principal
    linha.update(valores)  # Adicione os valores da linguagem
    linhas.append(linha)

# Criação do DataFrame
snipetsByLanguagePercentageDF = pd.DataFrame(linhas)
snipetsByLanguagePercentageDF = snipetsByLanguagePercentageDF.T

# Salvar o DataFrame
snipetsByLanguagePercentageDF = snipetsByLanguagePercentageDF.fillna(0)
snipetsByLanguagePercentageDF.to_csv('snipetsByLanguagePercentageDF.csv')